<a href="https://colab.research.google.com/github/Data-Creater-Atlas/Data-Atlas/blob/jinho/Mission_2_0917.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 및 경로 설정

In [17]:
!pip -q install ultralytics matplotlib opencv-python pandas

In [31]:
import os
from pathlib import Path
import json
import math
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

## Google Drive Mounting

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 절대 경로 설정

In [33]:
# ---- 경로 정의 ----
DATA_ROOT = "/content/drive/MyDrive/Data_Creater_Camp"

# !! 아래 4개의 경로는 사용자 환경에 맞게 수정하세요 !!
Train_Source_DIR = f"{DATA_ROOT}/Training/01.원천데이터/TS_KS"       # 원본 학습 이미지 폴더
Train_Label_DIR  = f"{DATA_ROOT}/Training/02.라벨링데이터/TL_KS_LINE"  # 원본 학습 라벨(JSON) 폴더
Validation_Source_DIR = f"{DATA_ROOT}/Validation/01.원천데이터/VS_KS"  # 원본 검증 이미지 폴더
Validation_Label_DIR  = f"{DATA_ROOT}/Validation/02.라벨링데이터/VL_KS_LINE"  # 원본 검증 라벨(JSON) 폴더

## 데이터 전처리 데이터셋 디렉토리
* crop 파일 X, index.csv/labels만 생성

In [34]:
DATASET_DIR = Path(DATA_ROOT) / "ResNet_Dataset"
for sub in ["train/images", "train/labels", "valid/images", "valid/labels"]:
    (DATASET_DIR / sub).mkdir(parents=True, exist_ok=True)

## 디바이스 설정

In [35]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

DEVICE: cuda


# 데이터 전처리 및 관리 (JSON 파싱 + 인덱싱)

In [36]:
def _safe_get(d: dict, key: str, default=None):
  return d[key] if (isinstance(d, dict) and key in d) else default

def load_line_labels(label_dir:str):
  label_dir = Path(label_dir)
  json_files = sorted(label_dir.glob("*.json"))
  items = []

  for jf in json_files:
    with open(jf, "r", encoding="utf-8") as f:
      data = json.load(f)

      if isinstance(data, dict):
        entries = list(data.values())
      elif isinstance(data, list):
        entries = data
      else:
        continue

      for entry in entries:
        filename = _safe_get(entry, "filename", None)
        regions = _safe_get(entry, "regions", [])
        if not filename or not regions:
          continue

        #각 region 마다 라인 혹은 polyline
        for region in regions:
          shape = _safe_get(region, "shape_attributes", {})
          attrs = _safe_get(region, "region_attributes", {})

          name = _safe_get(shape, "name", "").lower()
          x1 = y1 = x2 = y2 = None

          if name == "line":
            x1 = float(_safe_get(shape, "x", np.nan))
            y1 = float(_safe_get(shape, "y", np.nan))
            x2 = float(_safe_get(shape, "x2", np.nan))
            y2 = float(_safe_get(shape, "y2", np.nan))
          elif name == "polyline":
            xs = _safe_get(shape, "all_point_x", [])
            ys = _safe_get(shape, "all_point_y", [])
            if isinstance(xs, list) and isinstance(ys, list) and len(xs) >= 2 and len(ys) >= 2:
              x1, y1 = float(xs[0]), float(ys[0])
              x2, y2 = float(xs[-1]), float(ys[-1])

          else :
            continue

          if None in (x1, y1, x2, y2):
            continue

          # region_attributes에서 높이/ID 추출
          height = _safe_get(attrs, "height", None)
          chi_id = _safe_get(attrs, "chi_id", None)

          # 숫자형 변환(가능하면)
          try:
            height = float(height)
          except:
            # 높이가 없는 경우 스킵
            continue

          items.append({
                    "img_path": filename,  # 절대경로 변환/상대경로 변환은 export 단계에서 처리
                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                    "height": height,
                    "chi_id": chi_id
          })

  return items

In [37]:
def export_index_and_labels_only(items, image_dir: str, out_label_dir: str, out_index_csv: str, data_root: str):
  image_dir = Path(image_dir)
  out_label_dir = Path(out_label_dir)
  out_label_dir.mkdir(parents=True, exist_ok=True)
  rows = []

  # 이미지 파일명별 카운터
  per_image_counter = {}

  for it in items:
    img_file = Path(image_dir) / it["img_path"]
    if not img_file.exists():
      candidates = list(image_dir.rglob(Path(it["img_path"]).name))
      if candidates:
        img_file = candidates[0]
      else:
        print(f"이미지를 찾을 수 없습니다 {it['img_path']}")
        continue

    stem = img_file.stem
    per_image_counter[stem] = per_image_counter.get(stem, 0) + 1
    cnt = per_image_counter[stem]

    label_filename = f"{stem}_{cnt}.txt"
    # 라벨 파일 쓰기 (height만)
    with open(out_label_dir / label_filename, "w", encoding="utf-8") as f:
      f.write(str(it["height"]))

    # DATA_ROOT 기준 상대 경로로 저장
    data_root_path = Path(data_root)
    try:
        rel_img_path = str(img_file.relative_to(data_root_path))
    except Exception:
        # 상대경로 계산 실패 시 절대경로 저장(최후수단)
        rel_img_path = str(img_file)

    rows.append({
        "img_path": rel_img_path,
        "label_file": label_filename,
        "chi_id": it["chi_id"],
        "height": it["height"],
        "x1": it["x1"], "y1": it["y1"], "x2": it["x2"], "y2": it["y2"]
    })


## 학습 / 검증 인데스 생성

In [38]:
train_items = load_line_labels(Train_Label_DIR)
valid_items = load_line_labels(Validation_Label_DIR)

train_index_csv = DATASET_DIR / "train" / "index.csv"
valid_index_csv = DATASET_DIR / "valid" / "index.csv"

_ = export_index_and_labels_only(
    train_items, Train_Source_DIR, DATASET_DIR / "train" / "labels", train_index_csv, DATA_ROOT
)
_ = export_index_and_labels_only(
    valid_items, Validation_Source_DIR, DATASET_DIR / "valid" / "labels", valid_index_csv, DATA_ROOT
)

# PyTorch 데이터셋 / 데이터 로더

In [42]:
def crop_line_patch_np(img_bgr: np.ndarray, x1, y1, x2, y2, out_size=224) -> np.ndarray:
    """
    입력: BGR 이미지, 라인 좌표
    동작:
      1) 라인 중심과 각도 계산
      2) 이미지 전체를 -angle 만큼 회전시켜 라인이 수평이 되도록 함
      3) 회전된 이미지에서 (라인 중심) 기준 224x224 패치를 crop
      4) 경계 이슈가 있으면 BORDER_REFLECT로 보정
      5) RGB로 변환 후 반환
    """
    H, W = img_bgr.shape[:2]
    # 중심 및 각도
    cx = (x1 + x2) * 0.5
    cy = (y1 + y2) * 0.5
    angle_rad = math.atan2((y2 - y1), (x2 - x1))
    angle_deg = np.degrees(angle_rad)

    # 회전 행렬 (반시계 기준이므로 수평 정렬을 위해 -angle)
    M = cv2.getRotationMatrix2D((cx, cy), -angle_deg, 1.0)
    rotated = cv2.warpAffine(img_bgr, M, (W, H), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # 회전 후 중심점 좌표
    center = np.array([cx, cy, 1.0], dtype=np.float32)
    rcx, rcy = M @ center

    # crop 범위 계산
    half = out_size // 2
    x_min = int(round(rcx - half))
    y_min = int(round(rcy - half))
    x_max = x_min + out_size
    y_max = y_min + out_size

    # 경계 보정: 필요 시 padding 후 crop
    pad_left = max(0, -x_min)
    pad_top = max(0, -y_min)
    pad_right = max(0, x_max - rotated.shape[1])
    pad_bottom = max(0, y_max - rotated.shape[0])

    if any([pad_left, pad_top, pad_right, pad_bottom]):
        rotated = cv2.copyMakeBorder(
            rotated, pad_top, pad_bottom, pad_left, pad_right, borderType=cv2.BORDER_REFLECT
        )
        x_min += pad_left
        x_max += pad_left
        y_min += pad_top
        y_max += pad_top

    patch = rotated[y_min:y_max, x_min:x_max]  # BGR
    # 혹시 패치 크기가 어긋났다면 리사이즈
    if patch.shape[0] != out_size or patch.shape[1] != out_size:
        patch = cv2.resize(patch, (out_size, out_size), interpolation=cv2.INTER_LINEAR)

    # BGR -> RGB
    patch_rgb = cv2.cvtColor(patch, cv2.COLOR_BGR2RGB)
    return patch_rgb

In [43]:
class IndexCSVHeightDataset(Dataset):
    def __init__(self, index_csv: str, data_root: str, transform=None, forbid_aug=False):
        """
        index.csv를 읽어, on-the-fly로 라인 패치를 추출(검증셋: 증강금지).
        forbid_aug=True인 경우, transform에서 증강 요소가 있어도 실행 전 반드시 제거해야 함(여기서는 transform 구성 단계에서 보장).
        """
        self.df = pd.read_csv(index_csv)
        self.data_root = Path(data_root)
        self.transform = transform
        self.forbid_aug = forbid_aug

        # height 라벨 파일이 동일 폴더에 있지만, 값은 index.csv에 이미 존재(확인용) — 여기선 df['height']를 사용

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # 원본 이미지 로딩
        img_path_rel = row["img_path"]
        img_path_abs = self.data_root / img_path_rel
        img_bgr = cv2.imread(str(img_path_abs), cv2.IMREAD_COLOR)
        if img_bgr is None:
            raise FileNotFoundError(f"Image not found: {img_path_abs}")

        # 좌표
        x1, y1, x2, y2 = row["x1"], row["y1"], row["x2"], row["y2"]
        # 패치 추출 (RGB)
        patch_rgb = crop_line_patch_np(img_bgr, x1, y1, x2, y2, out_size=224)

        # PIL 변환
        pil_img = Image.fromarray(patch_rgb)

        if self.transform is not None:
            pil_img = self.transform(pil_img)

        # 타겟 (height)
        height = float(row["height"])
        target = torch.tensor([height], dtype=torch.float32)

        return pil_img, target  # (C,H,W), (1,)

# ======= TEST =======

In [46]:
# ===========================
# Quick Fix: index.csv 복구 & 재생성  (with tqdm progress bars)
# ===========================
!pip -q install tqdm

from pathlib import Path
import pandas as pd
import json, os, glob
from tqdm import tqdm  # NEW

print("DATA_ROOT  :", DATA_ROOT)
print("TRAIN IMG  :", Train_Source_DIR)
print("TRAIN JSON :", Train_Label_DIR)
print("VALID IMG  :", Validation_Source_DIR)
print("VALID JSON :", Validation_Label_DIR)
print("DATASET_DIR:", DATASET_DIR)

# 1) 경로 존재 여부 점검
for p in [Train_Source_DIR, Train_Label_DIR, Validation_Source_DIR, Validation_Label_DIR]:
    print(f"[exists] {p} ->", os.path.exists(p))

# 2) 라벨 JSON 파일 수(재귀) 점검
train_jsons = sorted(Path(Train_Label_DIR).rglob("*.json"))
valid_jsons = sorted(Path(Validation_Label_DIR).rglob("*.json"))
print(f"#train json: {len(train_jsons)} | #valid json: {len(valid_jsons)}")

# 3) index 생성 함수 (tqdm 추가)
def _safe_get(d: dict, key: str, default=None):
    return d[key] if (isinstance(d, dict) and key in d) else default

def load_line_labels_recursive(label_dir: str, desc="parse"):
    """
    기존 load_line_labels와 동일하지만 재귀 탐색 사용.
    tqdm 진행률 표시 추가.
    """
    items = []
    json_list = sorted(Path(label_dir).rglob("*.json"))
    for jf in tqdm(json_list, desc=f"[{desc}] JSON files", unit="file"):
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)
        entries = list(data.values()) if isinstance(data, dict) else (data if isinstance(data, list) else [])
        # region 단위 진행률 (entries 안에 regions가 있을 때만 대략적으로 표시)
        for entry in entries:
            filename = _safe_get(entry, "filename", None)
            regions = _safe_get(entry, "regions", [])
            if not filename or not regions:
                continue
            for region in regions:
                shape = _safe_get(region, "shape_attributes", {})
                attrs = _safe_get(region, "region_attributes", {})
                name = str(_safe_get(shape, "name", "")).lower()
                if name == "line":
                    x1 = float(_safe_get(shape, "x1", float("nan")))
                    y1 = float(_safe_get(shape, "y1", float("nan")))
                    x2 = float(_safe_get(shape, "x2", float("nan")))
                    y2 = float(_safe_get(shape, "y2", float("nan")))
                elif name == "polyline":
                    xs = _safe_get(shape, "all_points_x", [])
                    ys = _safe_get(shape, "all_points_y", [])
                    if not (isinstance(xs, list) and isinstance(ys, list) and len(xs) >= 2 and len(ys) >= 2):
                        continue
                    x1, y1, x2, y2 = float(xs[0]), float(ys[0]), float(xs[-1]), float(ys[-1])
                else:
                    continue
                if any(map(lambda v: v is None or (isinstance(v, float) and (pd.isna(v))), [x1,y1,x2,y2])):
                    continue
                height = _safe_get(attrs, "chi_height_m", None)
                try:
                    height = float(height)
                except:
                    continue
                items.append({
                    "img_path": filename,
                    "x1": x1, "y1": y1, "x2": x2, "y2": y2,
                    "height": height,
                    "chi_id": _safe_get(attrs, "chi_id", None)
                })
    return items

def export_index_and_labels_only(items, image_dir: str, out_label_dir: str, out_index_csv: str, data_root: str, desc="index"):
    """
    실제 크롭 이미지는 저장하지 않고, index.csv + height txt만 생성.
    tqdm 진행률 표시 추가.
    """
    from pathlib import Path
    import pandas as pd

    out_label_dir = Path(out_label_dir)
    out_label_dir.mkdir(parents=True, exist_ok=True)
    rows, per_image_counter = [], {}

    warn_missing = 0
    for it in tqdm(items, desc=f"[{desc}] build rows", unit="line"):
        img_file = Path(image_dir) / it["img_path"]
        if not img_file.exists():
            # 파일명이 경로와 불일치시 이미지 디렉토리 전체를 재귀 검색
            cands = list(Path(image_dir).rglob(Path(it["img_path"]).name))
            if cands:
                img_file = cands[0]
            else:
                # 마지막으로, DATA_ROOT 전역에서도 찾아보기
                cands2 = list(Path(data_root).rglob(Path(it["img_path"]).name))
                if cands2:
                    img_file = cands2[0]
                else:
                    warn_missing += 1
                    if warn_missing <= 10:
                        print(f"[WARN] image not found: {it['img_path']}")
                    continue

        stem = img_file.stem
        per_image_counter[stem] = per_image_counter.get(stem, 0) + 1
        cnt = per_image_counter[stem]
        label_filename = f"{stem}_{cnt}.txt"

        # height만 저장
        with open(out_label_dir / label_filename, "w", encoding="utf-8") as f:
            f.write(str(it["height"]))

        data_root_path = Path(data_root)
        try:
            rel_img_path = str(img_file.relative_to(data_root_path))
        except Exception:
            rel_img_path = str(img_file)

        rows.append({
            "img_path": rel_img_path,
            "label_file": label_filename,
            "chi_id": it["chi_id"],
            "height": it["height"],
            "x1": it["x1"], "y1": it["y1"], "x2": it["x2"], "y2": it["y2"],
        })

    if warn_missing > 10:
        print(f"[WARN] image not found (suppressed): {warn_missing - 10} more")

    df = pd.DataFrame(rows, columns=["img_path","label_file","chi_id","height","x1","y1","x2","y2"])
    Path(out_index_csv).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(out_index_csv, index=False, encoding="utf-8")
    print(f"Saved index csv: {out_index_csv} ({len(df)} rows)")
    return df

# 4) 실제 재생성 수행 (진행률 표시)
train_items = load_line_labels_recursive(Train_Label_DIR, desc="train-parse")
valid_items = load_line_labels_recursive(Validation_Label_DIR, desc="valid-parse")
print(f"Parsed items -> train: {len(train_items)}, valid: {len(valid_items)}")

train_index_csv = DATASET_DIR / "train" / "index.csv"
valid_index_csv = DATASET_DIR / "valid" / "index.csv"

_ = export_index_and_labels_only(
    train_items, Train_Source_DIR, DATASET_DIR / "train" / "labels", train_index_csv, DATA_ROOT, desc="train-index"
)
_ = export_index_and_labels_only(
    valid_items, Validation_Source_DIR, DATASET_DIR / "valid" / "labels", valid_index_csv, DATA_ROOT, desc="valid-index"
)

# 5) 생성 결과 미리보기
if train_index_csv.exists():
    print("\n[train/index.csv head]")
    display(pd.read_csv(train_index_csv).head())
if valid_index_csv.exists():
    print("\n[valid/index.csv head]")
    display(pd.read_csv(valid_index_csv).head())

DATA_ROOT  : /content/drive/MyDrive/Data_Creater_Camp
TRAIN IMG  : /content/drive/MyDrive/Data_Creater_Camp/Training/01.원천데이터/TS_KS
TRAIN JSON : /content/drive/MyDrive/Data_Creater_Camp/Training/02.라벨링데이터/TL_KS_LINE
VALID IMG  : /content/drive/MyDrive/Data_Creater_Camp/Validation/01.원천데이터/VS_KS
VALID JSON : /content/drive/MyDrive/Data_Creater_Camp/Validation/02.라벨링데이터/VL_KS_LINE
DATASET_DIR: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset
[exists] /content/drive/MyDrive/Data_Creater_Camp/Training/01.원천데이터/TS_KS -> True
[exists] /content/drive/MyDrive/Data_Creater_Camp/Training/02.라벨링데이터/TL_KS_LINE -> True
[exists] /content/drive/MyDrive/Data_Creater_Camp/Validation/01.원천데이터/VS_KS -> True
[exists] /content/drive/MyDrive/Data_Creater_Camp/Validation/02.라벨링데이터/VL_KS_LINE -> True
#train json: 8052 | #valid json: 1006


[valid-parse] JSON files: 100%|██████████| 1006/1006 [00:07<00:00, 125.79file/s]


Parsed items -> train: 10590, valid: 1323


[train-index] build rows: 100%|██████████| 10590/10590 [01:24<00:00, 125.47line/s]


Saved index csv: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/train/index.csv (10590 rows)


[valid-index] build rows: 100%|██████████| 1323/1323 [00:10<00:00, 126.89line/s]


Saved index csv: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/valid/index.csv (1323 rows)

[train/index.csv head]


,img_path,label_file,chi_id,height,x1,y1,x2,y2
0,Training/01.원천데이터/TS_KS/K3A_CHN_20161112052404...,K3A_CHN_20161112052404_0_1.txt,1,76.78,108.0,378.0,184.0,370.0
1,Training/01.원천데이터/TS_KS/K3A_CHN_20161112052404...,K3A_CHN_20161112052404_0_2.txt,2,63.81,221.0,402.0,284.0,394.0
2,Training/01.원천데이터/TS_KS/K3A_CHN_20161112052404...,K3A_CHN_20161112052404_1_1.txt,1,76.78,109.0,122.0,185.0,114.0
3,Training/01.원천데이터/TS_KS/K3A_CHN_20161112052404...,K3A_CHN_20161112052404_1_2.txt,2,64.81,221.0,146.0,285.0,138.0
4,Training/01.원천데이터/TS_KS/K3A_CHN_20161112052404...,K3A_CHN_20161112052404_10_1.txt,1,105.08,100.0,236.0,204.0,225.0



[valid/index.csv head]


,img_path,label_file,chi_id,height,x1,y1,x2,y2
0,Validation/01.원천데이터/VS_KS/K3A_CHN_201611120524...,K3A_CHN_20161112052404_15_1.txt,1,137.59,132.0,412.0,268.0,396.0
1,Validation/01.원천데이터/VS_KS/K3A_CHN_201701150511...,K3A_CHN_20170115051130_1_1.txt,1,108.90,389.0,112.0,457.0,101.0
2,Validation/01.원천데이터/VS_KS/K3A_CHN_201701230521...,K3A_CHN_20170123052151_1_1.txt,1,107.28,328.0,137.0,382.0,128.0
3,Validation/01.원천데이터/VS_KS/K3A_CHN_201701230521...,K3A_CHN_20170123052151_14_1.txt,1,99.56,444.0,132.0,494.0,123.0
4,Validation/01.원천데이터/VS_KS/K3A_CHN_201701230521...,K3A_CHN_20170123052151_22_1.txt,1,113.09,140.0,173.0,197.0,164.0


In [47]:
# ImageNet 정규화
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# 학습용: (필요시) 약한 증강 허용 — 검증셋은 별도로 augmentation 미적용 구성 사용
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),   # 허용
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# 검증용: 증강 금지 (표준화만)
valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

# ---- Dataset & DataLoader ----
train_ds = IndexCSVHeightDataset(index_csv=str(train_index_csv), data_root=DATA_ROOT, transform=train_transform, forbid_aug=False)
valid_ds = IndexCSVHeightDataset(index_csv=str(valid_index_csv), data_root=DATA_ROOT, transform=valid_transform, forbid_aug=True)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

len(train_ds), len(valid_ds)


(10590, 1323)

# ====== TEST ======

## Transforms

In [48]:
# # ImageNet 정규화
# IMAGENET_MEAN = [0.485, 0.456, 0.406]
# IMAGENET_STD  = [0.229, 0.224, 0.225]

# # 학습용: (필요시) 약한 증강 허용 — 검증셋은 별도로 augmentation 미적용 구성 사용
# train_transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.5),   # 허용
#     transforms.ToTensor(),
#     transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
# ])

# # 검증용: 증강 금지 (표준화만)
# valid_transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
# ])

# # ---- Dataset & DataLoader ----
# train_ds = IndexCSVHeightDataset(index_csv=str(train_index_csv), data_root=DATA_ROOT, transform=train_transform, forbid_aug=False)
# valid_ds = IndexCSVHeightDataset(index_csv=str(valid_index_csv), data_root=DATA_ROOT, transform=valid_transform, forbid_aug=True)

# train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
# valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

# print(f"Train size: {len(train_ds)}, Valid size: {len(valid_ds)}")


# 모델 학습(AMP) 및 평가 함수

In [52]:
ㅇ# 모델
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)

# 회귀 헤더 교체
in_features = model.fc.in_features
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(DEVICE)

# Optim, Loss
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 3e-4)

# AMP
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))

# Utilities
def rmse_from_mse(mse_val: float) -> float:
  return float(np.sqrt(mse_val))

def train_one_epoch(model, loader, optimizer, device=DEVICE, epoch=None):
    model.train()
    total_loss, n = 0.0, 0
    iterator = tqdm(loader,
                    desc=f"[train] epoch {epoch}" if epoch is not None else "[train]",
                    unit="batch", leave=False)

    for images, targets in iterator:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item() * images.size(0)
        n += images.size(0)

        # 진행 중 RMSE를 간단히 표시(옵션)
        cur_mse = total_loss / max(1, n)
        iterator.set_postfix(rmse=np.sqrt(cur_mse))

    mse = total_loss / max(1, n)
    rmse = float(np.sqrt(mse))
    return rmse

@torch.no_grad()
def evaluate(model, loader, device=DEVICE, desc="[valid]"):
    model.eval()
    preds_list, gts_list = [], []
    total_loss, n = 0.0, 0

    for images, targets in tqdm(loader, desc=desc, unit="batch", leave=False):
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        outputs = model(images)  # (B,1)
        loss = criterion(outputs, targets)
        total_loss += loss.item() * images.size(0)
        n += images.size(0)

        preds_list.append(outputs.detach().cpu().numpy())   # (B,1)
        gts_list.append(targets.detach().cpu().numpy())      # (B,1)

    mse = total_loss / max(1, n)
    rmse = float(np.sqrt(mse))
    preds = np.concatenate(preds_list, axis=0).reshape(-1)
    gts = np.concatenate(gts_list, axis=0).reshape(-1)
    return rmse, mse, preds, gts

class EarlyStopping:
    def __init__(self, patience=10, mode='min'):
        self.patience = patience
        self.mode = mode
        self.best = None
        self.num_bad = 0
        self.is_better = (lambda a, b: a < b) if mode == 'min' else (lambda a, b: a > b)

    def step(self, metric):
        if self.best is None or self.is_better(metric, self.best):
            self.best = metric
            self.num_bad = 0
            return True  # improved
        else:
            self.num_bad += 1
            return False  # not improved

/tmp/ipython-input-1892080553.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE.type == "cuda"))


## 모델 학습


In [53]:
MAX_EPOCHS = 80
PATIENCE = 10
BEST_CKPT = str(DATASET_DIR / "resnet18_lineheight_best.pt")

early = EarlyStopping(patience=PATIENCE, mode='min')
best_rmse = np.inf

for epoch in range(1, MAX_EPOCHS + 1):
    tr_rmse = train_one_epoch(model, train_loader, optimizer, DEVICE, epoch=epoch)
    va_rmse, va_mse, _, _ = evaluate(model, valid_loader, DEVICE, desc=f"[valid] epoch {epoch}")

    tqdm.write(f"[Epoch {epoch:03d}] train RMSE: {tr_rmse:.4f} | valid RMSE: {va_rmse:.4f}")

    if early.step(va_rmse):
        best_rmse = va_rmse
        torch.save(model.state_dict(), BEST_CKPT)
        tqdm.write(f"  ↳ New best! Saved checkpoint to: {BEST_CKPT}")
    else:
        if early.num_bad >= PATIENCE:
            tqdm.write(f"Early stopping triggered at epoch {epoch}. Best valid RMSE: {best_rmse:.4f}")
            break

[train] epoch 1:   0%|          | 0/331 [00:00<?, ?batch/s]/tmp/ipython-input-1892080553.py:33: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == "cuda")):


[Epoch 001] train RMSE: 89.5250 | valid RMSE: 53.0186
  ↳ New best! Saved checkpoint to: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/resnet18_lineheight_best.pt


[Epoch 002] train RMSE: 34.0763 | valid RMSE: 19.6312
  ↳ New best! Saved checkpoint to: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/resnet18_lineheight_best.pt


[Epoch 003] train RMSE: 18.0156 | valid RMSE: 16.5023
  ↳ New best! Saved checkpoint to: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/resnet18_lineheight_best.pt


[Epoch 004] train RMSE: 14.9557 | valid RMSE: 13.2361
  ↳ New best! Saved checkpoint to: /content/drive/MyDrive/Data_Creater_Camp/ResNet_Dataset/resnet18_lineheight_best.pt


...

[Epoch 039] train RMSE: 3.6388 | valid RMSE: 8.4351


[Epoch 040] train RMSE: 3.8846 | valid RMSE: 8.4890


[Epoch 041] train RMSE: 9.3956 | valid RMSE: 13.9103


[Epoch 042] train RMSE: 6.9914 | valid RMSE: 9.1856


[Epoch 043] train RMSE: 4.4184 | valid RMSE: 8.2162
Early stopping triggered at epoch 43. Best valid RMSE: 7.6239


# 성능 계산 (라인/이미지 라벨)

In [54]:
model.load_state_dict(torch.load(BEST_CKPT, map_location=DEVICE))
model.to(DEVICE)

final_rmse, final_mse, preds, gts = evaluate(model, valid_loader, DEVICE)
print(f"\n[Final] Line-level RMSE: {final_rmse:.4f}")

def rmse_image_level(preds: np.ndarray, gts: np.ndarray, index_csv_path: str):
    """
    검증 index.csv의 img_path를 기준으로 같은 이미지 내 여러 굴뚝(line)의
    예측/정답을 평균한 값으로 이미지 레벨 RMSE 계산.
    주의: DataLoader에서 shuffle=False여야 preds/gts 순서가 index.csv 순서와 일치.
    """
    df = pd.read_csv(index_csv_path)
    assert len(df) == len(preds) == len(gts), "Length mismatch between index.csv and predictions."

    df_eval = pd.DataFrame({
        "img_path": df["img_path"].values,
        "pred": preds,
        "gt": gts
    })
    grouped = df_eval.groupby("img_path").agg({"pred": "mean", "gt": "mean"}).reset_index()
    mse = np.mean((grouped["pred"].values - grouped["gt"].values) ** 2)
    rmse = float(np.sqrt(mse))
    return rmse

img_level_rmse = rmse_image_level(preds, gts, str(valid_index_csv))
print(f"[Final] Image-level RMSE: {img_level_rmse:.4f}")


[Final] Line-level RMSE: 7.6239
[Final] Image-level RMSE: 7.2201
